# Instalations

In [1]:
# !pip install psgtray
# !pip install pysimplegui
# !pip install keyboard

# Imports

In [2]:
from threading import Thread
import PySimpleGUI as sg
from psgtray import SystemTray
import keyboard

# Hotkeys

In [3]:
def hotkey_exec(window, i):
    window.write_event_value('-HOTKEY-', f'{i}')
    
    
def watch_keyboard(window):
    for i in range(0,10):
        keyboard.add_hotkey(f'ctrl + shift + {i}', hotkey_exec, args =(window, i))

def reset_keyboard():
    for i in range(0,10):
            keyboard.remove_hotkey(f'ctrl + shift + {i}')

# GUI

In [4]:
# Gerando elemento de cada botão
def gen_key_comp(i:int):
    i = i + 1 if i < 9 else 0 
    return sg.Frame(
        f'FUNC + {i}',
        [
        [sg.T(f'Flavor text of Func {i}', key=f'-{i}-', s=(15,2))], 
        [sg.Button('Edit', key=f'-EDIT{i}-')]
        ]
    )

# Gerando linhas de botões
gen_key_lines = lambda start, end: [gen_key_comp(i) for i in range(start, end)]
        


def main():
    # Estado da aplicação
    cur_mode = "Inativo"
    
    
    
    menu = [f'MODO: {cur_mode}', ['Show Window', 'Hide Window', '---', 'Exit']]
    tooltip = lambda x = cur_mode: f'Modo: {x}'

    layout = [
        gen_key_lines(0,5),
        gen_key_lines(5,10),
        [sg.Button('Exit')]
    ]

    window = sg.Window('PyHotKeys', layout, finalize=True, enable_close_attempted_event=True)
    
    
    
    watch_keyboard(window)
    tray = SystemTray(menu, single_click_events=False, window=window, tooltip=tooltip(), icon=sg.EMOJI_BASE64_HAPPY_JOY)
    tray.show_message('PyHotKeys', f'Iniciado, modo no modo: {cur_mode}')
    
    
    while True:
        event, values = window.read()

        if event == tray.key:
            event = values[event]

        if event in (sg.WIN_CLOSED, 'Exit'):
            break

        if event == '-HOTKEY-':
            tray.show_message(values[event])
        
        tray.show_message(title=event, message=values)

        
        
        if event in ('Show Window', sg.EVENT_SYSTEM_TRAY_ICON_DOUBLE_CLICKED):
            window.un_hide()
            window.bring_to_front()
        elif event in ('Hide Window', sg.WIN_CLOSE_ATTEMPTED_EVENT):
            window.hide()
            tray.show_icon()
            


    tray.close()            
    window.close()
    reset_keyboard()
    
if __name__ == '__main__':
    main()